In [ ]:
# default_exp auth

# auth

> python sdk for interacting with aws backend

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from nicHelper.wrappers import add_method
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
from nicHelper.dictUtil import printDict
from nicHelper.exception import errorString
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from typing import Optional

In [ ]:
#export
class FunctionName:
  def __init__(self, branch, appName):
    self.branch = branch; self.appName=appName
  def getName(self,function):
    return f'{self.appName}-{self.branch}-{function}'
    
class AuthSdk:
  def __init__(
    self,
    user = None,
    pw = None,
    sessionToken = None,
    region = 'ap-southeast-1',
    branch = 'dev-blank',
    appName = 'villa-employee'
    ):
    self.lambda_ = Lambda(user = user, pw = pw, sessionToken= sessionToken, region = region)
    self.functionName = FunctionName(branch, appName)
    
  def generalInvoke(self, body={}, function= 'unauth'):
    functionName = self.functionName.getName(function)
    event = Event.getInput(body=body)
    response = self.lambda_.invoke(functionName=functionName, input=event)
    try: return Response.parseBody(response)
    except: return response
  

In [ ]:
#hide
sdk = AuthSdk()
# sdk.lambda_.invoke(functionName = 'test', input = {'test':'test'})

In [ ]:
#export
@dataclass_json
@dataclass
class CreateCashier:
  user: str
  phone: str
  pw: str
  name: str
  cashierCode: str
  email: Optional[str] = None
    
@add_method(AuthSdk)
def createCashier(self,data={}, function= 'createCashier'):
  payload = CreateCashier(**data).to_dict()
  return self.generalInvoke(body=payload,function=function)

In [ ]:
sdk.createCashier({
  'user': 'nic1',
  'phone': '+66816684442',
  'pw': '12345678',
  'name': 'nic',
  'cashierCode': '001',
  'email': 'test@gmail.com'
})

{'success': True,
 'user': 'nic1',
 'phone': '+66816684442',
 'pw': '12345678',
 'name': 'nic',
 'cashierCode': '001',
 'email': 'test@gmail.com',
 'body': '{"error":"error updating An error occurred (AccessDeniedException) when calling the AdminUpdateUserAttributes operation: User: arn:aws:sts::394922924679:assumed-role\\/villaAuth-CreateCashierRole-VIHI2IVRWMHH\\/villa-employee-dev-blank-createCashier is not authorized to perform: cognito-idp:AdminUpdateUserAttributes on resource: arn:aws:cognito-idp:ap-southeast-1:394922924679:userpool\\/ap-southeast-1_QMI1IzkEx"}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
#export
@dataclass_json
@dataclass
class Auth:
  user: str
  password: str
@add_method(AuthSdk)
def auth(self,user,pw, function= 'auth'):
  payload = Auth(user=user, password=pw).to_dict()
  return self.generalInvoke(body=payload,function=function)

In [ ]:
result = sdk.auth(user='nic1',pw='12345678')
printDict(result)

AccessKeyId : ASIAVX4Z5T
SecretKey : HvnadpJUKc
SessionToken : IQoJb3JpZ2
Expiration : 1606998927.0
AccessToken : eyJraWQiOi
ExpiresIn : 3600
TokenType : Bearer
RefreshToken : eyJjdHkiOi
IdToken : eyJraWQiOi
NewDeviceMetadata
 DeviceKey : ap-southea
 DeviceGroupKey : -lt0cpemm
userInfo
 Username : nic1
 UserCreateDate : 1606995325.992
 UserLastModifiedDate : 1606995325.992
 Enabled : True
 UserStatus : CONFIRMED
 ResponseMetadata
  RequestId : eb486e0d-8
  HTTPStatusCode : 200
  HTTPHeaders
   date : Thu, 03 De
   content-type : applicatio
   content-length : 339
   connection : keep-alive
   x-amzn-requestid : eb486e0d-8
  RetryAttempts : 0
kwUserAttrib
 sub : 33cde0e8-6
 name : nic
 phone_number_verified : true
 phone_number : +668166844


## Confirm 
confirm phone/email of user by confirmation code

In [ ]:
#export
@dataclass_json
@dataclass
class Confirm:
  user: str
  code: str
@add_method(AuthSdk)
def confirm(self,user:str, code:str, function:str='confirm', **kwargs):
  payload = Confirm(user=user, code=code).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [ ]:
result=sdk.confirm('nic1', '123')
printDict(result)

error
 pickledb64string : gASVTQUAAA
 error : confirmati


In [ ]:
#export
@dataclass_json
@dataclass
class GetProfile:
  user: str
    
@add_method(AuthSdk)
def getProfile(self,user:str, function:str='getProfile', **kwargs):
  payload = GetProfile(user=user).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [ ]:
result=sdk.getProfile('nic1')
printDict(result)

Username : nic1
UserAttributes
 sub : 33cde0e8-6
 name : nic
 phone_number_verified : true
 phone_number : +668166844
UserCreateDate : 1606995325.992
UserLastModifiedDate : 1606995325.992
Enabled : True
UserStatus : CONFIRMED
ResponseMetadata
 RequestId : 45bdb065-0
 HTTPStatusCode : 200
 HTTPHeaders
  date : Thu, 03 De
  content-type : applicatio
  content-length : 339
  connection : keep-alive
  x-amzn-requestid : 45bdb065-0
 RetryAttempts : 0


In [ ]:
#export
@dataclass_json
@dataclass
class UpdateProfile:
  user: str
  attributes: dict
    
@add_method(AuthSdk)
def updateProfile(self,user:str, attributes:dict, function:str='updateProfile', **kwargs):
  payload = UpdateProfile(user=user, attributes=attributes).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [ ]:
result = sdk.updateProfile(**{
  'user': 'nic1',
  'attributes': {
    'custom:cashierCode': '1234'
  }
})
printDict(result)

Username : nic1
UserAttributes
 sub : 33cde0e8-6
 name : nic
 custom:cashierCode : 1234
 phone_number_verified : true
 phone_number : +668166844
UserCreateDate : 1606995325.992
UserLastModifiedDate : 1606995328.322
Enabled : True
UserStatus : CONFIRMED
ResponseMetadata
 RequestId : b2d28a55-6
 HTTPStatusCode : 200
 HTTPHeaders
  date : Thu, 03 De
  content-type : applicatio
  content-length : 384
  connection : keep-alive
  x-amzn-requestid : b2d28a55-6
 RetryAttempts : 0


In [ ]:
#export
@dataclass_json
@dataclass
class SetPassword:
  user: str
  pw: str
    
@add_method(AuthSdk)
def setPassword(self,user:str, pw:str, function:str='setPassword', **kwargs):
  payload = SetPassword(user=user, pw=pw).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [ ]:
sdk.setPassword(user='nic1',pw='12345678')

{'ResponseMetadata': {'RequestId': 'e535427f-52d0-4218-848e-c386e43dc058',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 03 Dec 2020 11:35:28 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e535427f-52d0-4218-848e-c386e43dc058'},
  'RetryAttempts': 0}}

In [ ]:
#export
@add_method(AuthSdk)
def unauth(self, function:str='unauth', **kwargs):
  return self.generalInvoke(body={}, function=function)

In [ ]:
result = sdk.unauth()
printDict(result)

AccessKeyId : ASIAVX4Z5T
SecretKey : +vyrk50YdD
SessionToken : IQoJb3JpZ2
Expiration : 1606998928.0


In [ ]:
#export
@dataclass_json
@dataclass
class DeleteUser:
  user: str
    
@add_method(AuthSdk)
def deleteUser(self,user:str, function:str='deleteUser', **kwargs):
  payload = DeleteUser(user=user).to_dict()
  return self.generalInvoke(body=payload, function=function)

In [ ]:
sdk.deleteUser(user='nic1')

{'ResponseMetadata': {'RequestId': '6c35e8ac-4426-4b06-82d5-528705afc783',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 03 Dec 2020 11:35:28 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6c35e8ac-4426-4b06-82d5-528705afc783'},
  'RetryAttempts': 0}}